In [ ]:
# Import packages
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
import cv2 as cv
from scipy import ndimage
import scipy.io as sio
from scipy.signal import butter, filtfilt
import skimage.measure

In [ ]:
# Check data
base_dir = 'C:\\Users\\Na Min An\\Desktop\\Development\\Fluorescence'
data_file = 'Fig3_ver01_221025_data119.cathodic.dF.dFF'
# 'Fig3_ver01_221025_data119.cathodic.dF.dFF'  # 10K
# 'Fig3_ver05_221114_data099.cathodic.dF,dFF' # 100K
# 'Fig3_ver05_221114_data101.anodic.dF,dFF' # 150K
# 'Fig3_ver05_221114_data239.cathodic.dF,dFF' # 50K
# 'Fig3_ver05_221114_data240.anodic.dF,dFF' # 60K
# 'Fig3_ver07_221123_data099data119.ImgSig' # W/o subtraction of darkframe values
# 'Fig3_ver08_221212_data211data213_picture,dF,dFF' # 15K 10K

data_dir = os.path.join(os.path.join(base_dir, 'data'), 'LightIntensity')
data_path = os.path.join(data_dir, data_file)
fig_dir = os.path.join(os.path.join(base_dir, 'figures'), 'LightIntensity')

test = sio.loadmat(data_path)
test

In [ ]:
# Check the background image
if data_file == 'Fig3_ver01_221025_data119.cathodic.dF.dFF':
    data_type = 'data119'
    background_path = 'C:\\Users\\Na Min An\\Desktop\\Development\\Fluorescence\\data\\LightIntensity\\Fig3_ver04_221110_data119.cathodic_picture.txt'
    low_thres, high_thres = 1.5*1e4, 2.5*1e4
elif data_file == 'Fig3_ver05_221114_data099.cathodic.dF,dFF':
    data_type = 'data099'
    background_path = 'C:\\Users\\Na Min An\\Desktop\\Development\\Fluorescence\\data\\LightIntensity\\Fig3_ver06_221114_data099.cathodic_picture.txt'
    low_thres, high_thres = 4.5*1e6, 6.25*1e6
elif data_file == 'Fig3_ver05_221114_data101.anodic.dF,dFF':
    data_type = 'data101'
    background_path = 'C:\\Users\\Na Min An\\Desktop\\Development\\Fluorescence\\data\\LightIntensity\\Fig3_ver06_221114_data101.anodic_picture.txt'
    low_thres, high_thres = 4.5*1e6, 6.25*1e6
elif data_file == 'Fig3_ver05_221114_data239.cathodic.dF,dFF':
    data_type = 'data239'
    background_path = 'C:\\Users\\Na Min An\\Desktop\\Development\\Fluorescence\\data\\LightIntensity\\Fig3_ver06_221114_data239.cathodic_picture.txt'
    low_thres, high_thres = 4.5*1e6, 6.25*1e6
elif data_file == 'Fig3_ver05_221114_data240.anodic.dF,dFF':
    data_type = 'data240'
    background_path = 'C:\\Users\\Na Min An\\Desktop\\Development\\Fluorescence\\data\\LightIntensity\\Fig3_ver06_221114_data240.anodic_picture.txt'
    low_thres, high_thres = 4.5*1e6, 6.25*1e6
elif data_file == 'Fig3_ver07_221123_data099data119.ImgSig':
    data_type = 'data099' # 'data119'
    background_path = 'C:\\Users\\Na Min An\\Desktop\\Development\\Fluorescence\\data\\LightIntensity\\Fig3_ver06_221114_data099.cathodic_picture.txt'
    # background_path = 'C:\\Users\\Na Min An\\Desktop\\Development\\Fluorescence\\data\\LightIntensity\\Fig3_ver04_221110_data119.cathodic_picture.txt'
    low_thres, high_thres = 4.5*1e6, 6.25*1e6
elif data_file == 'Fig3_ver08_221212_data211data213_picture,dF,dFF':
    data_type = 'data213' #  'data211' or 'data213'
    background_path = test['data213_anodic_picture'] # test['data211_cathodic_picture'] or test['data213_anodic_picture']
    low_thres, high_thres = 4.5*1e6, 6.25*1e6

if data_file == 'Fig3_ver01_221025_data119.cathodic.dF.dFF':
    kernel_dim = 3
else:
    kernel_dim = 6

os.makedirs(os.path.join(fig_dir, f'Frames_{data_type}'), exist_ok=True)

if data_file != 'Fig3_ver08_221212_data211data213_picture,dF,dFF':
    img_data = []
    with open(background_path) as file:
        for (i, line) in enumerate(file):
            row_data = np.expand_dims(np.array(line.split(',')[:-1]).astype('float64'), axis=0)
            if i == 0:
                img_data = row_data
            else:
                img_data = np.concatenate([img_data, row_data], axis=0)
else:
    img_data = background_path

plt.imshow(img_data, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
# Check keys
test.keys()

In [ ]:
# Print out shapes, min and max values
if data_file == 'Fig3_ver01_221025_data119.cathodic.dF.dFF':
    dF_var = 'data119_cathodic_dF'
    dFF_var = 'data119_cathodic_dFF'
elif data_file == 'Fig3_ver07_221123_data099data119.ImgSig':
    dF_var = 'data099_ImagSig'
    dFF_var = 'data119_ImgSig'
elif data_file == 'Fig3_ver08_221212_data211data213_picture,dF,dFF':
    dF_var = 'data213_anodic_dF' #'data211_cathodic_dF' # 'data213_anodic_dF'
    dFF_var = 'data213_anodic_dFF' #'data211_cathodic_dFF' # 'data213_anodic_dFF'
else:
    dF_var = 'dF'
    dFF_var = 'dFF'

print(test[dF_var].shape)
print(test[dF_var].min())
print(test[dF_var].max())

print(test[dFF_var].shape)
print(test[dFF_var].min())
print(test[dFF_var].max())

# The F0 value will be somewhere between these two below values.
print(test[dF_var].max() / test[dFF_var].min())
print(test[dF_var].min() / test[dFF_var].max())

In [ ]:
# Cut off the front frames

front_frame_num = 0
if data_file == 'Fig3_ver01_221025_data119.cathodic.dF.dFF':
    orig_data = test[dF_var][:, :, front_frame_num:] #-10]
else:
    orig_data = test[dF_var][:, :, front_frame_num:]

print(orig_data.shape)
print(orig_data.min())
print(orig_data.max())

In [ ]:
# Temporal processing

## Low-pass filtering (10 hz)
order = 5
cutoff = 10 # desired cutoff frequency of the filter, Hz
fs = 1000. # sampling rate, Hz
nyq = 0.5 * fs # Nyquist frequency
normal_cutoff = cutoff / nyq
b, a = butter(order, normal_cutoff, btype='low', analog=False)

temp_data = np.zeros(orig_data.shape)
for x_coord in range(orig_data.shape[0]):
    for y_coord in range(orig_data.shape[1]):
        filt_data = filtfilt(b, a, orig_data[x_coord, y_coord, :])
        temp_data[x_coord, y_coord, :] = filt_data

print(temp_data.shape)
print(temp_data.min())
print(temp_data.max())

In [ ]:
# Spatial processing

## Movie scaling
# space_data1_temp = np.swapaxes(temp_data, 0, -1)
# space_data1 = (space_data1_temp - space_data1_temp.min(axis=0)) / (space_data1_temp.max(axis=0) - space_data1_temp.min(axis=0))
# space_data1 *= 255
space_data1 = np.swapaxes(temp_data, 0, 2)

## Low-pass filtering (kernel x kernel mean)
iter_num = 3
kernel = np.ones((kernel_dim, kernel_dim)) 
space_data2 = np.zeros(space_data1.shape)
for frame_num in range(space_data2.shape[0]):
    space_data2[frame_num, :, :] = ndimage.convolve(space_data1[frame_num, :, :], kernel)
    for i in range(iter_num - 1):
        space_data2[frame_num, :, :] = ndimage.convolve(space_data2[frame_num, :, :], kernel)

## Subtracting the reference frame
space_data3 = np.zeros(space_data2.shape)
for x_coord in tqdm(range(space_data3.shape[1])):
    for y_coord in range(space_data3.shape[2]):
        baseline = np.median(space_data2[100:900, x_coord, y_coord])
        space_data3[:, x_coord, y_coord] =  space_data2[:, x_coord, y_coord] - baseline

print(space_data1.shape)
print(space_data1.min())
print(space_data1.max())
print(space_data2.shape)
print(space_data2.min())
print(space_data2.max())
print(space_data3.shape)
print(space_data3.min())
print(space_data3.max())

In [ ]:
# Visualize one frame for full processing steps
frame_num = 1500 # 1447 or frame_num

plt.imshow(test[dF_var][:, :, frame_num], cmap='gray')
plt.show()

plt.imshow(temp_data[:, :, frame_num], cmap='gray')
plt.show()

plt.imshow(space_data1[frame_num, :, :], cmap='gray')
plt.show()

plt.imshow(space_data2[frame_num, :, :], cmap='gray')
plt.show()

plt.imshow(space_data3[frame_num, :, :], cmap='gray')
plt.show()

In [ ]:
x_coord, y_coord = 65, 45   # 67, 47 # 65, 45 # 10, 10 

plt.plot(range(orig_data.shape[-1]), orig_data[x_coord, y_coord, :])
plt.xlabel('Time (us)')
plt.ylabel('Relative Light Intensity (Au)')
plt.show()

plt.plot(range(temp_data.shape[-1]), temp_data[x_coord, y_coord, :])
plt.xlabel('Time (us)')
plt.ylabel('Relative Light Intensity (Au)')
plt.show()

plt.plot(range(space_data1.shape[0]), space_data1[:, x_coord, y_coord])
plt.xlabel('Time (us)')
plt.ylabel('Relative Light Intensity (Au)')
plt.show()

plt.plot(range(space_data2.shape[0]), space_data2[:, x_coord, y_coord])
plt.xlabel('Time (us)')
plt.ylabel('Relative Light Intensity (Au)')
plt.show()

plt.plot(range(space_data3.shape[0]), space_data3[:, x_coord, y_coord])
plt.xlabel('Time (us)')
plt.ylabel('Relative Light Intensity (Au)')
plt.ylim([space_data3.min(), space_data3.max()])
plt.show()

space_data3.shape

- Generate a discrete colored mask

In [ ]:
colors = ['#1c0414', '#008cfd', '#3FFF00', '#ff1203']
color_lst = [np.array((28,4,20)), np.array((0, 140, 253)), np.array((63, 255, 0)), np.array((255, 18, 3))]

matplotlib.colors.ListedColormap(colors)

Baseline: average whole pixels

In [ ]:
mask = np.zeros(space_data3.shape)
colored_mask = np.zeros((mask.shape[0], 3, mask.shape[2], mask.shape[-1]))

max = space_data3.max()
min = space_data3.min()
one_interval = (max - min) / len(colors)

for x_coord in tqdm(range(space_data3.shape[1])):
    for y_coord in range(space_data3.shape[-1]):

        for frame_num in range(space_data3.shape[0]):

            signal_pnt = space_data3[frame_num, x_coord, y_coord]
            # if signal_pnt == max:
            #     bin_pnt = len(colors)-1
            # else:
            #     bin_pnt = (signal_pnt - min) // one_interval
            if signal_pnt >= (max/2):
                bin_pnt = 3
            elif signal_pnt < (min/2):
                bin_pnt = 1
            else:
                bin_pnt = 2

            # for images
            mask[frame_num, x_coord, y_coord] = bin_pnt
            # for video clips
            colored_mask[frame_num, :, x_coord, y_coord] = color_lst[int(bin_pnt)]


unique, counts = np.unique(mask, return_counts=True)
print("Colors: ", unique, "Counts: ", counts)

- Print images of separate masks

In [ ]:
matplotlib.rcParams['font.family'] = "sans-serif"

for frame_num in tqdm(range(0, space_data3.shape[0], 2)):
    data = mask[frame_num, :, :]
    if np.array_equal(np.unique(mask[frame_num, :, :], return_counts=True)[0], np.array([0., 1., 2., 3.])):
        colors = ['#000000', '#008cfd', '#3FFF00', '#ff1203']
    elif np.array_equal(np.unique(mask[frame_num, :, :], return_counts=True)[0], np.array([0., 1., 2.])):
        colors = ['#000000', '#008cfd', '#3FFF00']
    elif np.array_equal(np.unique(mask[frame_num, :, :], return_counts=True)[0], np.array([0., 1., 3.])):
        colors = ['#000000', '#008cfd', '#ff1203']
    elif np.array_equal(np.unique(mask[frame_num, :, :], return_counts=True)[0], np.array([1., 2., 3.])):
        colors = ['#008cfd', '#3FFF00', '#ff1203']
    elif np.array_equal(np.unique(mask[frame_num, :, :], return_counts=True)[0], np.array([0., 1.])):
        colors = ['#000000', '#008cfd']
    elif np.array_equal(np.unique(mask[frame_num, :, :], return_counts=True)[0], np.array([0., 2.])):
        colors = ['#000000', '#3FFF00']
    elif np.array_equal(np.unique(mask[frame_num, :, :], return_counts=True)[0], np.array([1., 2.])):
        colors = ['#008cfd', '#3FFF00']
    elif np.array_equal(np.unique(mask[frame_num, :, :], return_counts=True)[0], np.array([0.])):
        colors = ['#000000']
    elif np.array_equal(np.unique(mask[frame_num, :, :], return_counts=True)[0], np.array([1.])):
        colors = ['#008cfd']
    elif np.array_equal(np.unique(mask[frame_num, :, :], return_counts=True)[0], np.array([2.])):
        colors = ['#3FFF00']
    elif np.array_equal(np.unique(mask[frame_num, :, :], return_counts=True)[0], np.array([3.])):
        colors = ['#ff1203']

    fig, ax = plt.subplots(1)
    # ax.imshow(img_data[:, :], cmap='gray') # background
    ax.imshow(data, cmap=matplotlib.colors.ListedColormap(colors)) #, alpha=0.4)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f'Frame {frame_num+front_frame_num}', fontname='Georgia', fontsize=18)
    plt.tight_layout()
    plt.savefig(fig_dir + f'\\Frames_{data_type}\\{frame_num+front_frame_num}.png', bbox_inches='tight')
    
plt.close()

- Make a video clip of generated masks

In [ ]:
img_array_lst = []
for frame_num in tqdm(range(0, space_data3.shape[0], 2)):
    img = cv.imread(fig_dir + f'\\Frames_{data_type}\\{frame_num+front_frame_num}.png')
    h, w, _ = img.shape
    assert h == 470 and w == 441
    size = (w, h)
    img_array_lst.append(img)

out = cv.VideoWriter(fig_dir + f'\\{data_type}.mp4', cv.VideoWriter_fourcc(*'mp4v'), 15, (441, 470), True)

for i in range(space_data3.shape[0]):
    out.write(img_array_lst[i])
out.release()